#NECESSARY IMPORTS


In [ ]:
%%time

# For OCR
%pip install -q autocorrect
%pip install -q azure-cognitiveservices-vision-computervision
%pip install -q pillow


# For NLP
%pip install -q gensim
%pip install -q nltk
%pip install -q sklearn
%pip install -q tensorflow_hub
%pip install -q pyemd


# For Flask Application
# For Web Application
%pip install -q flask
%pip install -q flask_ngrok
%pip install -q werkzeug
%pip install -q flask_wtf
%pip install pyngrok
%pip install pot

%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while ge

In [ ]:
# For OCR
from autocorrect import Speller
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes, VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from getpass import getpass
from setup import *
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(azure_key))


# For NLP
import gensim
import gensim.downloader
import numpy as np
import pandas as pd
import tensorflow_hub as hub
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import download
from pyemd import emd
from sklearn.metrics.pairwise import cosine_similarity
download('stopwords')
stop_words = stopwords.words('english')

import os
import re
import sys
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# OCR code

In [ ]:
"""
Takes a list of uploaded filenames and returns list of file_text
"""
def get_ocr_text(uploaded_files):
    to_return = []

    if not uploaded_files:
        print("No files uploaded!")

    check = Speller(lang='en')
    for i, image in enumerate(uploaded_files, 1):
        # Open the image
        local_image_handwritten = open(image, "rb")

        # Call API with image and raw response (allows you to get the operation location)
        recognize_handwriting_results = computervision_client.read_in_stream(local_image_handwritten, raw=True)

        # Get the operation location (URL with ID as last appendage)
        operation_location_local = recognize_handwriting_results.headers["Operation-Location"]
        # Take the ID off and use to get results
        operation_id_local = operation_location_local.split("/")[-1]

        # Call the "GET" API and wait for the retrieval of the results
        while True:
            recognize_handwriting_result = computervision_client.get_read_result(operation_id_local)
            if recognize_handwriting_result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        # print(f"===== Extracted text from image #{i}: =====")
        lines = []
        # Print results, line by line
        if recognize_handwriting_result.status == OperationStatusCodes.succeeded:
            for text_result in recognize_handwriting_result.analyze_result.read_results:
                for line in text_result.lines:
                    # print(line.text, sep=' ')        # original OCR'ed line
                    # only autocorrect words which aren't abbreviations.
                    corrected = [word if bool(re.search("[A-Z]+", word)) else check(word) for word in line.text.split() ]
                    # print(" ".join(corrected))
                    lines.append(" ".join(corrected))

        to_return.append(" ".join(lines))
        # print()
    return to_return

# NLP Code

In [ ]:
def cos_sim(input_vectors):
    similarity = cosine_similarity(input_vectors)
    return similarity

negative = ["not" , "without", "against", "bad", "useless", "no", "dislike", "hate"]

def semantic_similarity(actual_answer , given_answer) :
    actual = actual_answer.lower().split(".")
    given = given_answer.lower().split(".")

    sim_checker = actual

    not_matching_semantics = list()

    semantic_1 = 0   # Actual_answer
    semantic_2 = 0   # Given_answer

    actual_embed_list = list()
    given_embed_list = list()

    for z in range(len(actual)) :
        list_actual = list()
        list_actual.append(actual[z])
        actual_embed_list.append(embed(list_actual))

    for z in range(len(given)) :
        semantic_1 = 0
        semantic_2 = 0
        list_given = list()
        list_given.append(given[z])
        embed_z = embed(list_given)

        sim_check = sim_checker.copy()
        sim_check.append(given[z])

        sen_em = embed(sim_check)

        similarity_matrix = cos_sim(np.array(sen_em))

        similarity_matrix_df = pd.DataFrame(similarity_matrix)

        cos_list = list(similarity_matrix_df[len(similarity_matrix_df) - 1])
        cos_list = cos_list[:len(cos_list)-1]

        index = cos_list.index(max(cos_list))

        actual_check = actual[index]
        actual_check = actual_check.split()
        for i in range(len(actual_check) - 1) :
            if(actual_check[i] in negative and actual_check[i+1] in negative) :
                semantic_1 += 1
            elif(actual_check[i] in negative and actual_check[i+1] not in negative) :
                semantic_1 -= 1

        answer_given = given[z].split()
        for i in range(len(answer_given) - 1) :
            if(answer_given[i] in negative and answer_given[i+1] in negative) :
                semantic_2 += 1
            elif(answer_given[i] in negative and answer_given[i+1] not in negative) :
                semantic_2 -= 1

        if(semantic_1 == 0 and semantic_2 == 0) :

            """
            Well and good
            """
        elif(semantic_1 < 0  and semantic_2 >= 0) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)

        elif(semantic_1 >= 0 and semantic_2 < 0 ) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)


        given_embed_list.append(embed_z)
    actual_embed = actual_embed_list[0]

    for i in range(len(actual_embed_list)-1) :

        actual_embed += actual_embed_list[i+1]

    given_embed = given_embed_list[0]
    for i in range(len(given_embed_list) - 1) :
        given_embed += given_embed_list[i+1]

    actual_embed = np.array(actual_embed).reshape(512)
    given_embed = np.array(given_embed).reshape(512)
    sem_checker = list([actual_embed,given_embed])
    answer = pd.DataFrame(cos_sim(sem_checker))

    return not_matching_semantics , answer[0][1]

In [ ]:
def WMD(actual_answer, given_answer, model) :
    actual_answer = actual_answer.lower().split()
    actual_answer = [w for w in actual_answer if w not in stop_words]

    given_answer = given_answer.lower().split()
    given_answer = [w for w in given_answer if w not in stop_words]

    return model.wmdistance(given_answer,actual_answer)

In [ ]:
def score(given_answer, actual_answer, model) :
    given_answer1 = given_answer[:]
    actual_answer1 = actual_answer[:]

    given_answer2 = given_answer[:]
    actual_answer2 = actual_answer[:]

    not_matching , similarity = semantic_similarity(actual_answer1, given_answer1)
    distance = WMD(actual_answer2, given_answer2, model)

    # if(similarity > 0) :
    # if(distance == 0) :
    #     return 1
    print("NOT MATCHING TEXT: ", not_matching)
    return similarity/distance
    # else :
        # return -1

# Run with Flask

In [ ]:

%cd /content/
!git clone https://github.com/Nileshmalav/Automatic-Answers-Checker.git
%cd Automatic-Answers-Checker
!printf "Currently in directory: "; pwd

# import flask and setup
from flask import Flask, redirect, render_template, request, make_response, jsonify
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
import os
from flask import Flask
from pyngrok import ngrok
# from openai import OpenAI

port_no = 5000
app= Flask(__name__)
ngrok. set_auth_token ("2YabLee4pluYgzJYudeDrhq7XSL_6T7LZU5m2C7GaXTkSatEv")
public_url = ngrok.connect(port_no).public_url

/content
Cloning into 'Automatic-Answers-Checker'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 66 (delta 14), reused 63 (delta 11), pack-reused 0
Receiving objects: 100% (66/66), 4.85 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/Automatic-Answers-Checker
Currently in directory: /content/Automatic-Answers-Checker


In [ ]:


UPLOAD_FOLDER = os.path.join("static","assets","img", 'uploads')
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

app.config['JSON_SORT_KEYS'] = False

def file_name(filename):
    return os.path.join(UPLOAD_FOLDER, filename)

@app.route("/")
@app.route("/index")
def home():
    return render_template("index.html")

@app.route("/live-demo")
def live_demo():
    return render_template("live-demo.html")

@app.route("/result", methods = ['GET', 'POST'])
def result():

    print("******* Inside RESULT **********")
    if request.method == 'POST':
        print("IT'S POST MALONE: ", request.files)

        teacher_file = request.files.get('file1', None)
        student_files = request.files.getlist('file2', None)

        print("Got files: ", teacher_file, student_files)

        if teacher_file and student_files:
            teacher_file.save(file_name(teacher_file.filename))

            for file_ in student_files:
                file_.save(file_name(file_.filename))

            teacher_text = get_ocr_text([file_name(teacher_file.filename)])[0]
            students_texts = get_ocr_text([file_name(f.filename) for f in student_files])
            print("OCR READING")
            # print(teacher_text)
            # print(students_texts)

            resulting_scores = [my_score(teacher_text, stud ) for stud in students_texts]
            print("res is of length", len(resulting_scores))

            # return redirect(url_for(""))
            results_dict = {
                "Teacher Filepath": file_name(teacher_file.filename),
                "Teacher Filename": teacher_file.filename,
                "Student Grades": [{
                    "Filepath": file_name(f.filename),
                    "Filename": f.filename,
                    "Score": r,
                } for f, r in zip(student_files, resulting_scores)],
            }
            return render_template("output.html", result=results_dict)
    return make_response("Invalid somehow!")#redirect(url_for('home'))

# @app.route("/results")
def output():
    pass

print(f"To acces the Gloable link please click {public_url}")
app.run(port=port_no)


To acces the Gloable link please click https://2815-34-80-139-67.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:51] "GET /static/assets/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:51] "GET /static/assets/js/theme.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:51] "GET /static/assets/fonts/ionicons.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:51] "GET /static/assets/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:51] "GET /static/assets/js/jquery.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:53] "GET /static/assets/fonts/ionicons.ttf?v=2.0.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2023 20:41:55] "GET /favicon.ico HTTP/1.1" 4

In [ ]:
%cd /content
%rm -r Automatic-Answers-Checker
%ls